In [1]:
import shapely as shp
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
accidents = pd.read_pickle('../../data/interim/features/accidents.pickle')
accidents['geometry'] = list(zip(accidents.lon, accidents.lat))
accidents['geometry'] = accidents.geometry.apply(shp.geometry.Point)
accidents = gpd.GeoDataFrame(accidents, geometry='geometry')

In [8]:
roads = pd.read_pickle('../../data/interim/features/roads.pickle')

In [28]:
roads.crs

{'proj': 'lcc',
 'lat_1': 29.3,
 'lat_2': 30.7,
 'lat_0': 28.5,
 'lon_0': -91.33333333333333,
 'x_0': 1000000,
 'y_0': 0,
 'datum': 'NAD83',
 'units': 'us-ft',
 'no_defs': True}

In [30]:
accidents.crs = {'init' :'epsg:4326'}

In [32]:
accidents = accidents.to_crs('+proj=lcc +lat_1=29.3 +lat_2=30.7 +lat_0=28.5 +lon_0=-91.33333333333333 +x_0=1000000 +y_0=0 +ellps=GRS80 +datum=NAD83 +to_meter=0.3048006096012192 +no_defs ')